In [67]:
!pip install beautifulsoup4 -q
from bs4 import BeautifulSoup as Soup
import requests 
import pandas as pd
import numpy as np

In [8]:
pid = 188550102
response = requests.get(f"https://www.safeway.com/shop/product-details.{pid}.html")

<!DOCTYPE HTML>
<html lang="en">
<head><script src="/Vnsane-to-done-I-set-I-praten-Gods-nor-thy-Mind-" async></script>
    <!-- <meta http-equiv="content-type" content="text/html; charset=UTF-8" -->
    <meta http-equiv="X-UA-Compatible" content="IE=edge; charset=UTF-8"/>

    
    <meta name="template" content="unified-productdetails-page"/>
    <meta name="robots" content="INDEX, FOLLOW"/>
    <meta http-equiv="Content-Security-Policy" content="default-src 'self' 'unsafe-inline' https://* 'unsafe-eval';
          img-src 'self' https://* data: ;
          style-src 'self' 'unsafe-inline' https://*;
          worker-src blob: ;"/>

     




	<title>Aidells Smoked Chicken Sausage Links Chicken & Apple 4 Count - 12 Oz - Safeway</title>


	<meta name="description" content="Shop Aidells Smoked Chicken Sausage Links Chicken & Apple 4 Count - 12 Oz from Safeway. Browse our wide selection of Dinner Sausage for Delivery or Drive Up & Go to pick up at the store!"/>


	<link rel="canonical" hr

In [109]:
def clean_text(text):
    idx = text.lower().find('amount per serving')
    if idx != -1:
        text = text[:idx] + text[idx + len('amount per serving'):]
    text = text.replace("(-)", "0")
    text = text.replace("-", "")
    text = text.strip()
    return text

def get_safeway_data_by_pid(pid):
    response = requests.get(f"https://www.safeway.com/shop/product-details.{pid}.html")    
    return response

def get_product_data_from_html_response(response):
    # Setup
    data = {}
    
    # Use BeautifulSoup to parse the response
    soup = Soup(response.text)

    # Get product name
    product_name = soup.find('title')
    data['Name'] = product_name.text.replace(' - Safeway', '')

    # Find html table with info on product
    table = soup.find('table', attrs={'class':'tableOfIngredients'})
    if not table:
        print('failed to find ingredients tab for: ', data['Name'])
        return False
    table_body = table.find('tbody')

    # get serving info
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td', attrs={'class':'table-ingredients-head body-s body-s--semi-bold'})
        if cols:
            serving_info = clean_text(cols[0].text).split('\n')
            spc_idx = serving_info[1].find('Servings Per Container: ')
            if spc_idx == -1:
                continue
            servings_per_container = serving_info[1][spc_idx + len('Servings Per Container: '):].strip()
            data["Servings Per Container"] = servings_per_container

    # get nutritional info
    rows = table_body.find_all('tr')
    for row in rows:
        header_cols = row.find_all('th')
        data_cols = row.find_all('td', attrs={'class':'table-ingredients-text'})
        if not header_cols and not data_cols:
            continue
        if header_cols:
            data[clean_text(header_cols[0].text)] = clean_text(data_cols[-2].text)
        elif clean_text(data_cols[1].text):
            data[clean_text(data_cols[0].text)] = clean_text(data_cols[-2].text)
        # skip the things that have only % Daily Values, not raw amounts

    return data

In [110]:
# Load valid_pids at this moment
valid_pids = np.load('valid_pids.npy')
print(valid_pids)

# Create dataframe to store safeway data
columns = ['Name', 'Servings Per Container', 'Calories', 'Calories From Fat', 'Total Fat', 'Saturated Fat', 'Trans Fat', 'Polyunsaturated Fat', 'Monounsaturated Fat', 'Cholesterol', 'Sodium', 'Total Carbohydrate', 'Dietary Fiber', 'Sugars', 'Protein', 'Vitamin A', 'Vitamin C', 'Calcium', 'Iron']
product_data = pd.DataFrame(columns=columns)

i = 0
while i < len(valid_pids):
    print(f'\r{i} ({pid})', flush=True, end="")
    pid = valid_pids[i]
    response = get_safeway_data_by_pid(pid)
    if response.status_code != 200:
        print("ERROR: invalid status code in response")
        continue
    data = get_product_data_from_html_response(response)
    if not data:
        i += 1
        continue
    product_data.loc[pid, :] = data
    # Save dataframe as we go
    if i % 10 == 0:
        product_data.to_csv('Safeway Product Data.csv')
    # Prep for next loop
    i += 1
    # Refresh valid_pids as we go - assumes we're updating valid pids in Safeway Data Sourcing simultaenously
    while i == len(valid_pids): 
        sleep(1)
        valid_pids = np.load('valid_pids.npy')


['960016747' '960127189' '188100136' ... '960194717' '960292719'
 '960272522']
0 (970012248)failed to find ingredients tab for:  Don Julio Tequila 1942 80 Proof - 750 Ml
1 (960016747)failed to find ingredients tab for:  Hennessy Cognac VS Very Special 80 Proof - 1.75 Liter
4 (960035907)failed to find ingredients tab for:  Remy Martin Champagne Cognac V.S.O.P - 750 Ml
6 (960518581)failed to find ingredients tab for:  Charmin Soft Mega Roll - 18 RL
7 (970012182)failed to find ingredients tab for:  Josh Cellars Wine Chardonnay - 750 Ml
8 (960063382)failed to find ingredients tab for:  Charmin Toilet Paper Ultra Soft Mega Rolls - 12 Roll
9 (970011733)failed to find ingredients tab for:  Charmin Ultr Strng Tlt Tissue 12 Mr - 12 RL
10 (970005270)failed to find ingredients tab for:  Charmin Toilet Tissue Ultra Strong Mega Roll - 18 Count
13 (960320799)failed to find ingredients tab for:  Ferrari-Carano Wine Classic Chardonnay Sonoma County - 750 Ml
15 (960239470)failed to find ingredients tab

143 (960137461)failed to find ingredients tab for:  MacRostie Chardonnay Wine - 750 Ml
144 (960044798)failed to find ingredients tab for:  Michelob Ultra Beer Cans - 18-12 Fl. Oz.
146 (960038466)failed to find ingredients tab for:  Allegra Allergy 24 Hour Non-Drowsy Tablets 180 mg - 70 Count
149 (960326658)failed to find ingredients tab for:  Firestone 805 In Cans - 12-12 Fl. Oz.
150 (960110761)failed to find ingredients tab for:  Rodney Strong Vineyards Wine Chardonnay California 2018 - 750 Ml
151 (189057057)failed to find ingredients tab for:  Chandon Sparkling Wine California Brut Classic Limited Edition - 750 Ml
156 (960260774)failed to find ingredients tab for:  St. Francis Wine Cabernet Sauvignon Sonoma County - 750 Ml
158 (960130440)failed to find ingredients tab for:  Apothic Cab Red Wine Cabernet Sauvignon California - 750 Ml
159 (960547077)failed to find ingredients tab for:  ROSIE Organic Chicken Whole Chicken Fresh - 4.50 LB
160 (960164102)failed to find ingredients tab for

285 (960056841)failed to find ingredients tab for:  Red Raspberries Prepacked - 12 Oz
287 (960315595)failed to find ingredients tab for:  Joel Gott Wine Pinot Gris Oregon - 750 Ml
288 (960080291)failed to find ingredients tab for:  Bread & Butter Wine Pinot Noir - 750 Ml
289 (960241080)failed to find ingredients tab for:  Duracell Battery Copper Top Alkaline AA - 24 Count
290 (960014565)failed to find ingredients tab for:  TEMPTATIONS Classic Cat Treats Crunchy And Soft Tasty Chicken Flavor - 30 Oz
294 (189011036)failed to find ingredients tab for:  Don Julio Tequila Anejo Claro 80 Proof - 750 Ml
295 (189031325)failed to find ingredients tab for:  Clos du Bois Wine White Sauvignon Blanc - 750 Ml
296 (189053327)failed to find ingredients tab for:  The Christian Brothers Brandy VS Very Smooth 80 Proof - 1.75 Liter
298 (960251469)failed to find ingredients tab for:  Roederer Brut Estate Wine - 750 Ml
299 (189100288)failed to find ingredients tab for:  St Francis Old Vine Zinfandel Wine - 

453 (960282696)failed to find ingredients tab for:  Open Nature Grass Fed Angus Boneless Ribeye Steak - .75 Lb.
454 (960079608)failed to find ingredients tab for:  Clos du Bois Wine Red Merlot - 750 Ml
456 (960106124)failed to find ingredients tab for:  Wente Wine Chardonnay Morning Fog Livermore Valley San Francisco Bay - 750 Ml
461 (188560019)failed to find ingredients tab for:  Gillette Fusion5 Mens Razor Blades - 4 Count
463 (960295720)failed to find ingredients tab for:  USDA Choice Beef Cross Rib Chuck Roast Boneless - 3.00 Lbs.
465 (237050086)failed to find ingredients tab for:  Tylenol Extra Strength Acetaminophen Caplets - 225 Count
467 (117200032)failed to find ingredients tab for:  ROCKY Chicken Breast Boneless Skinless Value Pack - 2.50 LB
468 (960452685)failed to find ingredients tab for:  Angel Soft Bathroom Tissue Mega Rolls White - 12 Roll
469 (960561764)failed to find ingredients tab for:  Blue Wilderness Adult Dog Chicken - 20 Lb
470 (960497050)failed to find ingredie

612 (189050367)failed to find ingredients tab for:  Michelob Ultra Beer Superior Light Slimcan - 12-12 Fl. Oz.
613 (189010937)failed to find ingredients tab for:  Chateau St. Jean Wine Chardonnay Creamy 2016 - 750 Ml
614 (960300134)failed to find ingredients tab for:  Qupe Central Coast Syrah Wine - 750 Ml
615 (189061366)failed to find ingredients tab for:  Smoking Loon Steelbird Chardonnay Wine - 750 Ml
616 (960075450)failed to find ingredients tab for:  Fresh Cut Grapes Mixed - 24 Oz
617 (960113634)failed to find ingredients tab for:  Matanzas Creek Wine White Sauvignon Blanc Alexander Valley - 750 Ml
619 (960021862)failed to find ingredients tab for:  Swiffer WetJet Mopping Pads Refill Multi Surface - 24 Count
622 (136050129)failed to find ingredients tab for:  Charmin Ultra Strong Toilet Tissue 6 Mega Roll - 6 RL
623 (970006976)failed to find ingredients tab for:  Natural Light Beer Cans - 30-12 Fl. Oz.
624 (189011046)failed to find ingredients tab for:  Peets Coffee Coffee Ground 

...

9462 (148011159)failed to find ingredients tab for:  Olmeca Altos Tequila Reposado 80 Proof - 750 Ml
9465 (102010159)failed to find ingredients tab for:  Sonic Ptb Optimal Clean Replacement Bh - 3 Count
9466 (960350757)failed to find ingredients tab for:  Dry Idea Advanced Dry Antiperspirant Deodorant Unscented Roll-On - 3.25 Fl. Oz.
9467 (960016885)failed to find ingredients tab for:  Coca-Cola Soda Pop Classic - 355 Ml
9469 (960070106)failed to find ingredients tab for:  Signature Care Cold & Flu Relief Nighttime Acetaminophen 325mg Softgel - 24 Count
9474 (127102948)failed to find ingredients tab for:  Progresso Traditional Soup 99% Fat Free Chicken Noodle - 19 Oz
9478 (960050991)failed to find ingredients tab for:  Scone Blueberry 8 Ct
9479 (960540970)failed to find ingredients tab for:  Ghirardelli Chocolate Squares Dark Chocolate Assortment - 14.86 Oz
9484 (960533137)failed to find ingredients tab for:  Signature Care Naproxen Sodium 220mg Rain Reliever Fever Reducer NSAID Tablet

9685 (960168811)failed to find ingredients tab for:  Icy Hot Medicated Patch Back And Large Areas - 5 Count
9687 (117100066)failed to find ingredients tab for:  California Sun Dry Tomatoes Julienne In Oil - 16 Oz
9688 (960169462)failed to find ingredients tab for:  Fancy Feast Cat Food Dry Ocean Fish & Salmon And Accents Of Garden Greens - 3 Lb
9689 (132070179)failed to find ingredients tab for:  Jose Cuervo Sparkling Margarita Rtd - 4-335 Ml
9692 (960053746)failed to find ingredients tab for:  Ling Ling Teriyaki Chicken Potsticker - 24 Oz
9693 (960332603)failed to find ingredients tab for:  Pedigree Dog Food Adult Wet Chopped Ground Dinner Variety Pack - 18-3.5 Oz
9697 (960189984)failed to find ingredients tab for:  Dove Style+Care Hairspray Flexible Hold - 7 Fl. Oz.
9702 (105400004)failed to find ingredients tab for:  Persil ProClean Laundry Detergent Liquid 2in1 - 100 Fl. Oz.
9705 (124250173)failed to find ingredients tab for:  Oral B Charcoal Manual Toothbrush Soft - 2 CT
9707 (960

9893 (960282626)failed to find ingredients tab for:  Stove Top Stuffing Mix Cornbread Box - 6 Oz
9895 (148010622)failed to find ingredients tab for:  Belgioioso Asiago Fresco Snacking Bag - 18 Oz
9900 (138250341)failed to find ingredients tab for:  Good Cook Pot Holder Glove - Each
9901 (960103857)failed to find ingredients tab for:  Seafood Service Counter Shrimp Cooked 51-60 Count Medium Previously Frozen - 1.00 LB
9902 (960267482)failed to find ingredients tab for:  Pantene Pro V Repair & Protect Shampoo - 12.6 Fl. Oz.
9904 (960040773)failed to find ingredients tab for:  LavAzza Coffee Ground Dark Roast Intenso - 12 Oz
9905 (960225531)failed to find ingredients tab for:  TRESemme Hair Mousse Flawless Curls Extra Hold - 10.5 Oz
9912 (960254103)failed to find ingredients tab for:  Ancient Peaks Cabernet Sauvignon Wine - 750 Ml
9913 (960039563)failed to find ingredients tab for:  Signature SELECT Soup Condensed Cream of Chicken - 10.5 Oz
9914 (125010007)failed to find ingredients tab f

10114 (109550065)failed to find ingredients tab for:  Boars Head Cheese Wheel Manchego - 0.5 Lb
10119 (960272115)failed to find ingredients tab for:  Systane Eye Drops Ultra Lubricant Unit Dose 25 Count - .7 Ml
10120 (960157927)failed to find ingredients tab for:  Dove Men+Care Body + Face Wash Extra Fresh - 13.5 Oz
10122 (960040203)failed to find ingredients tab for:  Baskin Robbins Ice Cream Peanut Butter N Chocolate - 14 Fl. Oz.
10143 (960027509)failed to find ingredients tab for:  Fresh Baked Susan Cookies 24 Count
10145 (184710354)failed to find ingredients tab for:  Napa Cellars Sauvignon Blanc Wine - 750 Ml
10148 (960073687)failed to find ingredients tab for:  Fort Point Animal In Cans - 6-12 Fl. Oz.
10149 (960478521)failed to find ingredients tab for:  Clean & Clear Deep Action Exfoliating Oil Free Scrub - 5 Oz
10165 (960042186)failed to find ingredients tab for:  BIC Lighter Multi Purpose Classic Edition - Each
10168 (960099413)failed to find ingredients tab for:  Neutrogena O

10376 (970009949)failed to find ingredients tab for:  Imodium Multi-Symptom Relief Caplets - 18 Count
10377 (960052261)failed to find ingredients tab for:  Benefiber Fiber Supplement Sugar-Free Powder - 8.7 Oz
10378 (960147723)failed to find ingredients tab for:  Good Cook Clips Mini Bag - 2 Count
10380 (960051613)failed to find ingredients tab for:  Mars Fun Size Variety Mix Stand Up Pouch - 31.18 OZ
10388 (125010036)failed to find ingredients tab for:  Near East Rice Pilaf Mix Spanish Rice Box - 6.75 Oz
10391 (960100942)failed to find ingredients tab for:  Reynolds Wrap Aluminum Foil 30 Sq. Ft. - Each
10393 (960542069)failed to find ingredients tab for:  Signature SELECT Cereal Bran Flakes - 17.3 Oz
10398 (960194717)failed to find ingredients tab for:  Birds Eye Veggie Made Cauliflower Mashed Roasted Garlic - 12 Oz
10399 (960292719)failed to find ingredients tab for:  RITZ Toasted Chips 40% Less Fat Cheddar - 8.1 Oz


In [111]:
product_data

,Name,Servings Per Container,Calories,Calories From Fat,Total Fat,Saturated Fat,Trans Fat,Polyunsaturated Fat,Monounsaturated Fat,Cholesterol,Sodium,Total Carbohydrate,Dietary Fiber,Sugars,Protein,Vitamin A,Vitamin C,Calcium,Iron
188100136,Ground Beef 93% Lean 7% Fat Value Pack- 3.5 Lbs.,varied,170,70,8g,3g,0g,0,0,40mg,60mg,0g,0g,0g,23g,,,,
960035907,Red Bull Energy Drink Sugar Free - 12-8.4 Fl. Oz.,12,5,0,0g,0,0g,0,0,0,105mg,3g,0,0g,<1g,,,,
960518581,Enfagrow Premium Milk Toddler Next Step Nutrit...,About 25,160,0,6g,2.5g,0g,0,0,5mg,80mg,20g,<1g,0,6g,,,,
137050304,Tillamook Sharp Cheddar Cheese Loaf - 32 Oz,32,120,0,10g,6g,0g,0,0,30mg,200mg,1g,0g,0,6g,,,,
960320799,Enfamil NeuroPro Infant Formula Milk Based Pow...,NaN,100,0,0,0,0,0,0,0,27mcg,0,0,0,2g,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960542069,O Organics Quinoa Tri Color - 16 Oz,about 10,150,0,2g,0g,0g,0,0,0mg,0mg,31g,3g,0,5g,,,,
117100036,Gold Medal Flour Whole Wheat - 5 Lb,About 75,100,0,0.5g,0g,0g,0g,0g,0mg,0mg,21g,3g,0,4g,,,,
960042371,Signature SELECT Soda Diet Cola Caffeine Free ...,12,0,0,0g,0,0,0,0,0,10mg,0g,0,0g,0g,,,,
960072031,Morton McCormick Iodized Salt & Pepper Shaker ...,75,0,0,0g,0,0,0,0,0,590mg,0g,0,0,0g,,,,
